## statsmodels

In [1]:
import pyhdfe
import numpy as np
import statsmodels.api as sm
from sklearn import datasets

pyhdfe.__version__

C:\Programs\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


'0.1.1'

In this tutorial, we'll use the [boston data set](https://scikit-learn.org/0.15/modules/generated/sklearn.datasets.load_boston.html) from [scikit-learn](https://scikit-learn.org/stable/) to demonstrate how pyhdfe can be used to absorb fixed effects before running regressions with [statsmodels](https://www.statsmodels.org/stable/regression.html). We'll also demonstrate how pyhdfe can be used to compute degrees of freedom used by fixed effects.

First, load the data set and create a matrix of fixed effect IDs. We'll use a dummy for the Charles river and an index of accessibility to radial highways.

In [2]:
boston = datasets.load_boston().data
ids = boston[:, [3, 8]]
ids

array([[0., 1.],
       [0., 2.],
       [0., 2.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

Next, choose our variables: per capita crime rate, proportion of residential land zoned for lots over 25,000 square feet, and proportion of non-retail business acres per town.

In [3]:
variables = boston[:, :3]
variables

array([[6.3200e-03, 1.8000e+01, 2.3100e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01],
       [1.0959e-01, 0.0000e+00, 1.1930e+01],
       [4.7410e-02, 0.0000e+00, 1.1930e+01]])

The :func:`create` function initializes an :class:`Algorithm` for fixed effect absorption that can residualize matrices with :meth:`Algorithm.residualize`. We'll use the default algorithm. You may want to try other algorithms if it takes a long time to absorb fixed effects into your data.

In [4]:
algorithm = pyhdfe.create(ids)
residualized = algorithm.residualize(variables)
residualized

array([[-1.08723516e-01, -2.20167195e+01, -2.65583593e+00],
       [-5.59754167e-02, -2.04166667e+01, -2.56083333e+00],
       [-5.59954167e-02, -2.04166667e+01, -2.56083333e+00],
       ...,
       [-5.42835164e-02, -4.00167195e+01,  6.96416407e+00],
       [-5.45351644e-03, -4.00167195e+01,  6.96416407e+00],
       [-6.76335164e-02, -4.00167195e+01,  6.96416407e+00]])

We can now run a regression of per capita crime rate on the other two variables and our fixed effects.

In [5]:
y = residualized[:, [0]]
X = residualized[:, 1:]
ols = sm.OLS(y, X)
result = ols.fit()
result.params

array([-6.97058632e-05,  5.53038164e-02])

Standard errors can be adjusted to account for the degrees of freedom that are lost because of the fixed effects. By default, fixed effect degrees of freedom are computed when :func:`create` initializes an algorithm and are stored in :attr:`Algorithm.degrees`.

In [6]:
se = result.HC0_se
se

array([0.00109298, 0.00962226])

In [7]:
se_adjusted = np.sqrt(np.square(se) * result.df_resid / (result.df_resid - algorithm.degrees))
se_adjusted

array([0.00110398, 0.00971916])